# **Session #6 Homework**

### **Loading Required Libraries**

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor, export_text, export_graphviz
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

import xgboost as xgb


import warnings
warnings.filterwarnings('ignore')

### **Getting the data**

- using the New York City Airbnb open Data

- Use only the following columns:
        'neighbourhood_group',
        'room_type',
        'latitude',
        'longitude',
        'minimum_nights',
        'number_of_reviews','reviews_per_month',
        'calculated_host_listings_count',
        'availability_365',
        'price'
- Fill NAs with 0
- Apply the log tranform to price
- Do train/validation/test split with 60%/20%/20% distribution.
- Use the train_test_split function and set the random_state parameter to 1
- Use DictVectorizer to turn the dataframe into matrices

In [ ]:
columns = [
    'neighbourhood_group', 'room_type', 'latitude', 'longitude',
    'minimum_nights', 'number_of_reviews','reviews_per_month',
    'calculated_host_listings_count', 'availability_365',
    'price'
]

df = pd.read_csv('AB_NYC_2019.csv', usecols=columns)
df.reviews_per_month = df.reviews_per_month.fillna(0)
df['price'] = np.log1p(df['price'])

In [8]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = df_train.price.values
y_val = df_val.price.values
y_test = df_test.price.values

del df_train['price']
del df_val['price']
del df_test['price']

In [9]:
#use DictVectorizer to turn train and validation into matrices:

dv = DictVectorizer(sparse=False)

df_train_dict = df_train.to_dict(orient='records')
df_val_dict = df_val.to_dict(orient='records')

X_train = dv.fit_transform(df_train_dict)
X_val = dv.transform(df_val_dict)

### **Question 1**

Let's train a decision tree regressor to predict the price variable.

   - Train a model with max_depth=1

In [10]:
model1 = DecisionTreeRegressor(max_depth=1)

model1.fit(X_train, y_train)



DecisionTreeRegressor(ccp_alpha=0.0, criterion='mse', max_depth=1,
                      max_features=None, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, presort='deprecated',
                      random_state=None, splitter='best')

Which feature is used for splitting the data?

   - room_type
   - neighbourhood_group
   - number_of_reviews
   - reviews_per_month

In [11]:
print(export_text(model1, feature_names=dv.get_feature_names()))

|--- room_type=Entire home/apt <= 0.50
|   |--- value: [4.29]
|--- room_type=Entire home/apt >  0.50
|   |--- value: [5.15]



### **Question 2**

Train a random forest model with these parameters:

   - n_estimators=10
   - random_state=1
   - n_jobs=-1 (optional - to make training faster)

What's the RMSE of this model on validation?

   - 0.059
   - 0.259
   - 0.459
   - 0.659


In [12]:
model2 = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
model2.fit(X_train, y_train)
pred = model2.predict(X_val)
rmse = mean_squared_error(y_val, pred, squared=False)
print(rmse)

0.460888788095762


### **Question 3**

Now let's experiment with the n_estimators parameter

   - Try different values of this parameter from 10 to 200 with step 10
   - Set random_state to 1
   - Evaluate the model on the validation dataset

After which value of n_estimators does RMSE stop improving?

   - 10
   - 50
   - 70
   - 120


In [13]:
for n in np.linspace(10,200, 20).astype(int):
    model3= RandomForestRegressor(n_estimators = n, random_state = 1, n_jobs = -1)
    model3.fit(X_train, y_train)
    predic = model3.predict(X_val)
    rmse = mean_squared_error(y_val, predic, squared=False)
    print(f"n_estimators = {n},    rmse = {rmse}")

n_estimators = 10,    rmse = 0.460888788095762
n_estimators = 20,    rmse = 0.44744055425901735
n_estimators = 30,    rmse = 0.44440015830200136
n_estimators = 40,    rmse = 0.44281711871165336
n_estimators = 50,    rmse = 0.44176815762157207
n_estimators = 60,    rmse = 0.44125165338475053
n_estimators = 70,    rmse = 0.44064455072014125
n_estimators = 80,    rmse = 0.440740830263047
n_estimators = 90,    rmse = 0.4402749020311618
n_estimators = 100,    rmse = 0.4397631903425602
n_estimators = 110,    rmse = 0.4393128261208466
n_estimators = 120,    rmse = 0.43912990819267467
n_estimators = 130,    rmse = 0.43927965755127013
n_estimators = 140,    rmse = 0.4391514983746916
n_estimators = 150,    rmse = 0.43902892484355854
n_estimators = 160,    rmse = 0.4388423640549406
n_estimators = 170,    rmse = 0.4387460509023033
n_estimators = 180,    rmse = 0.4388528948224223
n_estimators = 190,    rmse = 0.43879074353852315
n_estimators = 200,    rmse = 0.4387871286025359


### **Question 4**

Let's select the best max_depth:

   - Try different values of max_depth: [10, 15, 20, 25]
   - For each of these values, try different values of n_estimators from 10 till 200 (with step 10)
   - Fix the random seed: random_state=1

What's the best max_depth:

   - 10
   - 15
   - 20
   - 25


In [14]:
results = []
for depth in [10,15,20,25]:
    for n in np.linspace(10,200, 20).astype(int):
        model4= RandomForestRegressor(n_estimators = n, max_depth = depth, random_state = 1, n_jobs = -1)
        model4.fit(X_train, y_train)
        preds = model4.predict(X_val)
        rmse = mean_squared_error(y_val, predic, squared=False)
        results.append((depth, n, rmse))


In [15]:
df_results = pd.DataFrame(results, columns=['max_depth', 'n_estimators', 'rmse'])

In [17]:
df_results.max_depth[df_results.rmse.argmin()]


10

Bonus question (not graded):

Will the answer be different if we change the seed for the model?

In [ ]:
results1 = []
for depth in [10,15,20,25]:
    for n in np.linspace(10,200, 20).astype(int):
        model4= RandomForestRegressor(n_estimators = n, max_depth = depth, random_state = 2, n_jobs = -1)
        model4.fit(X_train, y_train)
        preds = model4.predict(X_val)
        rmse = mean_squared_error(y_val, predic, squared=False)
        results1.append((depth, n, rmse))


### **Question 5**

We can extract feature importance information from tree-based models.

At each step of the decision tree learning algorith, it finds the best split. When doint it, we can calculate "gain" - the reduction in impurity before and after the split. This gain is quite useful in understanding what are the imporatant features for tree-based models.

In Scikit-Learn, tree-based models contain this information in the feature_importances_ field.

For this homework question, we'll find the most important feature:

  * Train the model with these parametes:
     - n_estimators=10,
     - max_depth=20,
     - random_state=1,
     - n_jobs=-1 (optional)
  * Get the feature importance information from this model

What's the most important feature?

   * neighbourhood_group=Manhattan
   * room_type=Entire home/apt
   * longitude
   * latitude




In [19]:
model5 = RandomForestRegressor(n_estimators = 10, max_depth = 20, random_state = 1, n_jobs = -1)
model5.fit(X_train, y_train)
imp = model5.feature_importances_

modelImp = pd.DataFrame(imp, index = dv.feature_names_)
print(modelImp)


                                          0
availability_365                   0.076516
calculated_host_listings_count     0.030906
latitude                           0.152054
longitude                          0.153219
minimum_nights                     0.053645
neighbourhood_group=Bronx          0.000284
neighbourhood_group=Brooklyn       0.001166
neighbourhood_group=Manhattan      0.034017
neighbourhood_group=Queens         0.001153
neighbourhood_group=Staten Island  0.000118
number_of_reviews                  0.043525
reviews_per_month                  0.052438
room_type=Entire home/apt          0.391897
room_type=Private room             0.004521
room_type=Shared room              0.004541


### **Question 6**

Now let's train an XGBoost model! For this question, we'll tune the eta parameter

   - Install XGBoost
   - Create DMatrix for train and validation
   - Create a watchlist
   - Train a model with these parameters for 100 rounds:
   
 xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}


Now change eta first to 0.1 and then to 0.01
   
   
What's the best eta?

   - 0.3
   - 0.1
   - 0.01


In [22]:
features = dv.get_feature_names()
dftrain = xgb.DMatrix(X_train, label = y_train, feature_names = features)
dfval = xgb.DMatrix(X_val, label = y_val, feature_names = features)
watchlist = [(dftrain, 'train'), (dfval, 'val')]

xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}


In [24]:
model6 = xgb.train(xgb_params, dftrain, num_boost_round = 100, verbose_eval = 5, evals=watchlist)
y_pred = model6.predict(dfval)
rmse1 = mean_squared_error(y_val, y_pred, squared=False)


xgb_params = {
    'eta': 0.1, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
print("eta: 0.1")

[0]	train-rmse:3.02752	val-rmse:3.02415
[5]	train-rmse:0.674904	val-rmse:0.677523
[10]	train-rmse:0.439151	val-rmse:0.449813
[15]	train-rmse:0.423031	val-rmse:0.439126
[20]	train-rmse:0.417245	val-rmse:0.437226
[25]	train-rmse:0.412681	val-rmse:0.436174
[30]	train-rmse:0.407563	val-rmse:0.435124
[35]	train-rmse:0.404793	val-rmse:0.434324
[40]	train-rmse:0.401326	val-rmse:0.434602
[45]	train-rmse:0.399043	val-rmse:0.434132
[50]	train-rmse:0.395527	val-rmse:0.433357
[55]	train-rmse:0.391777	val-rmse:0.433642
[60]	train-rmse:0.388646	val-rmse:0.43327
[65]	train-rmse:0.383917	val-rmse:0.432713
[70]	train-rmse:0.380675	val-rmse:0.432784
[75]	train-rmse:0.377159	val-rmse:0.432945
[80]	train-rmse:0.374531	val-rmse:0.432804
[85]	train-rmse:0.37268	val-rmse:0.4331
[90]	train-rmse:0.370312	val-rmse:0.433563
[95]	train-rmse:0.366663	val-rmse:0.433719
[99]	train-rmse:0.36437	val-rmse:0.434066
eta: 0.1


In [25]:
model7 = xgb.train(xgb_params, dftrain, num_boost_round = 100, verbose_eval = 5, evals=watchlist)
y_pred1 = model7.predict(dfval)
rmse2 = mean_squared_error(y_val, y_pred1, squared=False)


xgb_params = {
    'eta': 0.01, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
print("eta: 0.01")

[0]	train-rmse:3.87217	val-rmse:3.86889
[5]	train-rmse:2.31905	val-rmse:2.31689
[10]	train-rmse:1.41919	val-rmse:1.41798
[15]	train-rmse:0.912897	val-rmse:0.913599
[20]	train-rmse:0.645011	val-rmse:0.648887
[25]	train-rmse:0.517091	val-rmse:0.52394
[30]	train-rmse:0.461519	val-rmse:0.471089
[35]	train-rmse:0.438045	val-rmse:0.450283
[40]	train-rmse:0.427864	val-rmse:0.44178
[45]	train-rmse:0.422597	val-rmse:0.438206
[50]	train-rmse:0.418963	val-rmse:0.436207
[55]	train-rmse:0.416481	val-rmse:0.435422
[60]	train-rmse:0.414688	val-rmse:0.434822
[65]	train-rmse:0.412339	val-rmse:0.434327
[70]	train-rmse:0.410824	val-rmse:0.433942
[75]	train-rmse:0.409584	val-rmse:0.433725
[80]	train-rmse:0.407816	val-rmse:0.433406
[85]	train-rmse:0.406555	val-rmse:0.433194
[90]	train-rmse:0.405447	val-rmse:0.432963
[95]	train-rmse:0.404238	val-rmse:0.43287
[99]	train-rmse:0.404032	val-rmse:0.432873
eta: 0.01


In [26]:
model8 = xgb.train(xgb_params, dftrain, num_boost_round = 100, verbose_eval = 5, evals=watchlist)
y_pred2 = model8.predict(dfval)
rmse3 = mean_squared_error(y_val, y_pred2, squared=False)

[0]	train-rmse:4.25336	val-rmse:4.2501
[5]	train-rmse:4.04779	val-rmse:4.04454
[10]	train-rmse:3.85242	val-rmse:3.84922
[15]	train-rmse:3.66674	val-rmse:3.66359
[20]	train-rmse:3.4903	val-rmse:3.48718
[25]	train-rmse:3.32263	val-rmse:3.31956
[30]	train-rmse:3.16332	val-rmse:3.16027
[35]	train-rmse:3.01196	val-rmse:3.00898
[40]	train-rmse:2.86817	val-rmse:2.86532
[45]	train-rmse:2.73158	val-rmse:2.72884
[50]	train-rmse:2.60185	val-rmse:2.59924
[55]	train-rmse:2.47865	val-rmse:2.47614
[60]	train-rmse:2.36167	val-rmse:2.35927
[65]	train-rmse:2.25061	val-rmse:2.24836
[70]	train-rmse:2.14519	val-rmse:2.14303
[75]	train-rmse:2.04515	val-rmse:2.04309
[80]	train-rmse:1.95021	val-rmse:1.94831
[85]	train-rmse:1.86016	val-rmse:1.85836
[90]	train-rmse:1.77473	val-rmse:1.77307
[95]	train-rmse:1.69373	val-rmse:1.69216
[99]	train-rmse:1.63197	val-rmse:1.63049


In [27]:
print(f"final validation rmse for ema = 0.3, rmse = {rmse1}")
print(f"final validation rmse for ema = 0.1, rmse = {rmse2}")
print(f"final validation rmse for ema = 0.01, rmse = {rmse3}")

final validation rmse for ema = 0.3, rmse = 0.4340655350827198
final validation rmse for ema = 0.1, rmse = 0.43287301324031663
final validation rmse for ema = 0.01, rmse = 1.6304939442439164


#                                                 **DONE**